In [ ]:
# Please execute this cell (shift+<Return>) before starting the workbook
# this should print out "Your notebook is ready to go"
import sys
import tskit
import pandas as pd
import numpy as np
import networkx as nx
import random

if "pyodide" in sys.modules:
    import tqdm
    import micropip
    await micropip.install('jupyterquiz')
    await micropip.install('seaborn')
    await micropip.install('scipy')
    await micropip.install('drawsvg')
    await micropip.install('pyyaml')
import scipy
import model
from model import wf
from model import coal
from model.draw import plot_ancestry
import workshop
workbook = workshop.setup_coalescent_handson()
display(workbook.setup)

# A hands-on introduction to the coalescent


In this exercise the goal is to gain an understanding of how the coalescent works by recreating the algorithm step by step. 

## Some basic properties of the coalescent

Before we start generating coalescent trees, we recapitulate some basic properties and terminology. 



<img src="img/coalescent.svg" width=300  />

The figure above shows a *coalescent tree* or a *genealogy*. The black circles are called the *nodes* of the tree and correspond to chromosomes, or genes. The dotted lines highlight time slices $T_i$ that correspond to the *waiting time* for a coalescent event when there are $i$ chromosomes. The waiting time is [*exponentially distributed*](https://en.wikipedia.org/wiki/Exponential_distribution) with intensity $\lambda=\frac{i(i-1)}{2}$ (but see note on time scales below), such that the expected waiting time when there are $i$ uncoalesced chromosome is 

$$
E(T_i) = \frac{2}{i(i-1)}
$$

As an example, in this time scale, $E(T_2)=1$. Using the waiting times $T_i$ we can infer the branch lengths, represented by $\tau_j$ in the illustration. For instance, $\tau_4=T_5$, whereas $\tau_2=T_5 + T_4$, and so on.

The *total tree height*, or the time to the most recent common ancestor (MRCA) $T_{MRCA}$, is the sum of the waiting times $T_i$, or

$$
T_{MRCA} = \sum_{i=2}^n T_i
$$

with expected value

$$
E(T_{MRCA}) = \sum_{i=2}^nE(T_i) = 2\sum_{i=2}^n\frac{1}{j(j-1)} = 2\left(1 - \frac{1}{n}\right)
$$

which converges to $E(T_{MRCA})=2$ as $n\rightarrow\infty$. This means that since $E(T_2)=1$, on average a coalescent process spends half of its time waiting for the two last chromosomes to coalesce. A corollary to this is that the return on investment of adding new samples diminishes quickly in terms of adding time to the MRCA.

In [ ]:
workbook.question("tmrca")

Finally, one can show that the expected *total tree length* is

$$
E(T_{total}) = \sum_{i=2}^n iE(T_i) = \sum_{i=2}^n i\frac{2}{i(i-1)} = 2\sum_{i=2}^n\frac{1}{i-1}
$$

<div class="alert alert-block alert-info"><b>A note on time scales</b>
    <br/>
   
The basic results regarding some simple properties of coalescent trees may differ according to the choice of time scale. The Wright-Fisher model uses *discrete* generations, where the average time it takes for two chromosome to find a common ancestor is 2N generations. It may however be conceptually and computationally advantageous to employ a *continuous time approximation*, in which one unit of time is some factor of the population size 2N (the average time for two chromosomes to coalesce). Some treatments use N or 4N as scaling factor, which lead to a factor 2 difference in some of the results. 
    
For instance, the coalescent simulator ms scales time in units of 4N, whereas msprime, which we will look at in the next exercise, measures time in generations. For more information on different ways of scaling, see
 <a href="https://tskit.dev/msprime/docs/stable/switch_from_other_simulators.html#scaled-parameters">msprime documentation on scaled parameters</a>.

    
In what follows, one unit of time is equivalent to 2N generations. This has the appealing property that the expressions only depend on the number of chromosomes at a given time step.</div>

## On mutations

Whereas coalescent waiting times are exponentially distributed, mutations occur according to a [*Poisson process*](https://en.wikipedia.org/wiki/Poisson_distribution) with intensity $t\theta/2$, where $t$ is the elapsed time, and $\theta$ is the *population mutation parameter*. Since the expected value of a Poisson random variable is equal to its intensity, the expected number of mutations $M_t$ occurring during time $t$ is 

$$
E(M_t) = \frac{t\theta}{2}
$$

This has important implications. To begin with, the expected number of mutations is proportional to time, which translated to a tree means the number of mutations are proportional to branch length.
Furthermore, the total expected number of mutations $M_T$ for a coalescent tree with total tree branch length $T_{total}$ is

$$
M_T = \frac{T_{total}\theta}{2}
$$

Knowing the parameter $\theta$ and the total tree branch length $T_{total}$, we can then sample a number of mutations from a Poisson distribution with intensity $\theta T_{total}/2$ and sprinkle the mutations on the branches of the tree.

# Pen and paper coalescent

We will start by tracing out the coalescent with pen and paper, only using the workbook to generate draws from the exponential or Poisson distribution. Make sure to take your time going through each step so you understand what is going on.


## Creating the genealogy

Let's create a genealogy for $n=4$ individuals. The algorithm to create a genealogy is as follows (taken from Hahn 2018):

<b>Algorithm: genealogy creation</b>
<ol>
    <li>Start with $i=n$ chromosomes</li>
    <li>Choose time to next coalescent event from an exponential distribution with parameter $\lambda=i(i-1)/2$</li>
    <li>Choose two chromosomes at random to coalesce</li>
    <li>Merge the two lineages and set $i \rightarrow i - 1$</li>
    <li>If $i>1$, go to step 2; if not, stop.</li>
</ol>

Although the algorithm describes the outline for generating a genealogy, an implementation needs to do a little book-keeping. 

Step 2 implies that we need to keep track of the elapsed time looking backwards somehow (call it "age"). Starting from time 0, all $n$ samples (leaves) have age 0, the first coalescent chromosome age $T_n$, the second coalescent chromosome age $T_n + T_{n-1}$, and so on. Individual branch lengths $\tau_i$ connecting child $i$ with its parent can then be obtained as the age of the parent minus the age of the child.

Step 3 implies we need to keep track of the chromosomes that haven't coalesced yet. Whenever two nodes coalesce into a parent, the children are removed from the remaining list of chromosomes and the parent id added to it.

<dl class="exercise"><dt>Exercise 1</dt>
    <dd>Draw four circles next to each other and label them 1-4. Simulate the waiting time with the code below and randomly choose two chromosomes to coalesce. Draw a new circle at a distance proportional to the waiting time above the four circles, label it 5, and connect it with the two coalescing chromosomes. Repeat the procedure by generating a new waiting time and selecting two chromosomes from label 5 and the two uncoalesced start chromosomes, draw a new circle labelled 6 above the four circles and connect to the coalescing chromosomes. Repeat the procedure one last time.
    </dd>
    <dd>
    
In order to facilitate placement of mutations, I recommend you also keep track of the individual branch lengths (equivalent to the time separating a child from its parent). For instance, you could let $\tau_i$ denote the length of the branch that leads from node $i$ to its ancestor.
    </dd>
</dl>

In [ ]:
# Exercise 1
# Simulate exponential variable for different values of uncoalesced chromosomes.
# NB: scipy.stats uses the so-called *scale* parametrization beta=1/lambda  
lmbda = 6
scipy.stats.expon(scale=1/lmbda).rvs()

In [ ]:
workbook.question("exponential_draw")

When you're done, you should have something similar to the genealogy shown below (note however that your genealogy will have a different topology and different branch lengths due to the stochastic nature of coalescent waiting times and ordering).

In [ ]:
workbook.draw("coalescent_tree")

In this example, the branch length connecting nodes 2 and 4 could be called $\tau_2$ (note that that length equals that of $\tau_3$), and so on. The total tree branch length $T_{total}$ is then the sum of all individual branches $\tau_i$.

## Adding mutations

Once you have generated a genealogy, it is time to add mutations. We will generate a number of mutations as a Poisson random variable and "throw" the mutations onto branches, where the probability of ending up on a particular branch is proportional to its length with respect to the total tree branch length $T_{total}$. Since we have a simulated tree with known branch lengths $\tau_i$ (and hence total tree length), we can *condition* on $T_{total}$ and choose a value of $\theta$ to generate an expected number of mutations $S$. 

The total number of mutations is Poisson distributed with mean $T_{total}\theta/2$, where $\theta=4N_e\mu$ is the *population mutation rate*.

In [ ]:
workbook.question("total_branch_length")

Let's choose $\theta$ so the expected number of mutations is $S=5$.

In [ ]:
workbook.question("theta_S5")

<dl class="exercise"><dt>Exercise 2</dt>
    <dd>Now that you have $T_{total}$ and $\theta$ we can simulate $S$, the number of mutations or the number of <em>segregating sites</em> (remember, we have chosen $\theta$ such that the expected number of segregating sites is 5). Also, using the individual branch lengths $\tau_i$ as weights, for each mutation we can simulate what branch to put it on.

Fill in the values for $T_{total}$, $\theta$, and the vector of branch lengths below and run the code.
    </dd>
    </dl>

In [ ]:
# Exercise 2
# Set the values below to the ones you obtained from your simulation
T_total =
theta =
# Recall: branches hold branch lengths (tau) where the array indices
# correspond to node ids
p = np.array(branches)
# Normalize values by sum to get probabilities
p = p / np.sum(p)
# Simulate S mutations based on the values of theta and T_total
S = scipy.stats.poisson(theta*T_total/2).rvs()
# NB: We "flatten" the value return by the multinomial to make 
# it return a list where the value at each position i corresponds
# to the number of drawn mutations on the branch from node i
mutations = scipy.stats.multinomial(n=S, p=p).rvs().flatten()
mutations, S

If all goes well, you should get a list that tells you how many mutations fall on what branch, and as sanity check the number of segregating sites $S$. In your drawing, mark with 'x' the mutations on each branch. You should end up with something like the illustration in the following code block, where (unordered) mutations are indicated as black rectangles on the branches.


In [ ]:
workbook.draw("coalescent_tree_w_mutations")

Try to derive the sequences of your simulated samples. In case you need some input on how to do so, complete the questions below.

In [ ]:
workbook.question("sample_sequences")

<div class="alert alert-block alert-info"><b>A note on the order of mutations</b>
    <br/>
In the simulation above, there is no explicit ordering of mutations along a genetic sequence. Therefore, when deriving the sequences at the tips, one could simply follow the rectangles from left to right, such that the sequence for sample <tt>1</tt> would be <tt>110000</tt>. 
    
It would be easy to improve the procedure above by, for instance, looping over the mutations to make one multinomial draw at a time and labelling mutations by order of occurrence on a genetic sequence.

</div>

# Coded coalescent

In this section, we will code the coalescent using python. Briefly, we need to transfer the manual steps from above into code. You will be asked to write two functions building on existing code snippets. And don't worry if python is not your language of choice, there are prepared functions that you can use as is. I still urge you to think through what code would be needed, and of course, don't hesitate to ask your teachers!

## Simulating the ancestry

We begin by writing a function called `sim_ancestry` whose goal is to simulate genealogies. The input should simply be the number of *samples*, which is equivalent to the number of leaves. We also need to know how many nodes and edges there are in a tree.

In [ ]:
workbook.question("nodes_in_tree")

In [ ]:
workbook.question("edges_in_tree")

Based on the information above, we could store the node ids in a list `nodes`, ranging from 0 to the number nodes you just derived. Then, the first $n$ entries correspond to the samples (the leaves in the tree), and the entries from position $n+1$ and onwards are internal nodes that correspond to ancestral chromosomes. We also need a list `ancestors` to store the ancestor of a node; this list can be one entry shorter than `nodes` since the root node by definition has no ancestor. Finally, we need a list `branches` that stores branch lengths, where an entry at index `i` corresponds to the branch length starting at the entry in `nodes[i]` (child) with the entry in `ancestors[i]` (parent).


<dl class="exercise"><dt>Exercise 3</dt>
    <dd>
        Complete the function <tt>sim_ancestry</tt> below and run some simulations. The function will return a list <tt>ancestors</tt> that stores the ancestor of node $i$ at position $i$ in the list. <tt>branches</tt> is the list of branch lengths $\tau_i$. 
        
There is a working version of the function in the <tt>coal</tt> module which you can run as, e.g., 
        
```python
coal.sim_ancestry(10)
```
        
</dd>
</dl>

In [ ]:
# Exercise 3. Complete the function below.
def sim_ancestry(samples):
    # Fill in the length of the nodes, ancestors, branches,
    # and age vectors. The argument to range is an integer.
    nodes = list(range( )) # range(number) produces numbers from 0 to number
    ancestors = list(range( ))
    branches = list(range( ))
    age = [0] * (len(  )) # len(list) is the length of a list
    # Keep list of indices that haven't coalesced
    # At start equal to list of samples
    uncoalesced = list(range(samples))
    i = samples
    current_time = 0
    while i > 1:
        lmbda = i*(i-1)/2
        t = scipy.stats.expon(scale=1/lmbda).rvs()
        # update current_time
        # current_time = 
        # Generate a parent node index
        parent = max(uncoalesced) + 1
        age[parent] = current_time
        # Sample children from list of uncoalesced ids and set their ancestor to parent
        child1, child2 = random.sample(uncoalesced, 2)
        ancestors[child1] = parent
        ancestors[child2] = parent
        # child1 and child2 have coalesced so remove from list
        uncoalesced.remove(child1)
        uncoalesced.remove(child2)
        # Add the ancestor of child1 and child2
        uncoalesced.append(parent)
        i = i - 1
    # Calculate branch lengths from node j as the age of the parent 
    for j in nodes[0:len(ancestors)]:
        branches[j] = age[ancestors[j]] - age[j]
    return ancestors, branches

Once you have a function that hopefully works, you can generate the  `ancestors` and `branches` lists. There is a utility function in `model.draw` that lets you draw the ancestry (see below). Make some simulations of varying sample sizes and plot the results.

In [ ]:
ancestors, branches = sim_ancestry(5)
plot_ancestry(ancestors, branches, width=400, height=500, 
              node_size=3, node_labels=True, jitter_label=(0, 20),
             show_internal=False)

<div class="alert alert-block alert-info"><b>Hint on plotting multiple trees</b>
    <br/>
    If you don't care about the contents of <tt>ancestors</tt> and <tt>branches</tt> and just want to visually compare multiple simulations, you could use the following code which makes use of <a href="https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#widget-list">Jupyter Widgets</a>
       
```python 
import ipywidgets as widgets
kwargs = dict(width=400, height=500, 
    node_size=3, node_labels=True,
    jitter_label=(0, 20), 
    show_internal=False, ylim=(0, 3))
widgets.HBox((
    widgets.HTML(plot_ancestry(*sim_ancestry(10), **kwargs).as_svg()), 
    widgets.HTML(plot_ancestry(*sim_ancestry(10), **kwargs).as_svg())
))
```   

Here, <tt>kwargs</tt> is a <a href="https://docs.python.org/3/tutorial/datastructures.html#dictionaries">python dictionary</a> that holds the options to <tt>plot_ancestry</tt>.
    
</div>

## Simulating mutations

Now that we have simulated a genealogy represented by the lists `ancestors` and `branches`, we can write a function to simulate a number of mutations according to a Poisson distribution. The code will be very similar to what we used in the manual example.

<dl class="exercise"><dt>Exercise 4</dt>
    <dd>Write a function called <tt>sim_mutations</tt> that takes as input the branch lengths <tt>branches</tt> and the population mutation rate $\theta$. The function should return a list `mutations` where each index entry contains the number of mutations on branch $i$.
        
As for the ancestry simulation, there is a function in the <tt>coal</tt> module that you can run as, e.g.,

```python
coal.sim_ancestry(branches, theta=3)
```
      
</dd>
</dl>    

In [ ]:
# Exercise 4
# Rename / complete where necessary
def functionname():
    tau = np.array(branches)
    # Calculate T_total by summing up the branch lengths
    Ttot = 
    # Generate branch probabilities 
    p = tau / 
    # Draw the number of segregating sites; insert parameter to poisson
    S = scipy.stats.poisson(...).rvs()
    # Generate multinomial draw of mutations
    mutations = scipy.stats.multinomial(n=S, p=p).rvs().flatten()
    # return the mutations

Once you have generated mutations for your genealogy, you can plot the resulting tree, now adding a list of mutations:


In [ ]:
plot_ancestry(ancestors, branches, mutations)

## Conclusions


Congratulations! You have now performed coalescent simulations by hand and drafted your own functions to perform the simulations. You could continue by adding functions to calculate summary statistics and so on, but we will refrain from doing so here, as the most important goal was to understand the basics of simulations. To perform more advanced operations, we will turn to [msprime](https://tskit.dev/msprime/docs/stable/intro.html) in the following lecture and exercise.
